# Hi :)

----

# Import libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# Initialization

In [3]:
# Scaler
scaler= StandardScaler()

# Dataset and DataLoader
class CNNFeatureDataset(Dataset):
    def __init__(self, X, y):
        self.X= torch.tensor(X, dtype= torch.float32)
        self.y= torch.tensor(y, dtype= torch.long)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Define 1D CNN
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.network= nn.Sequential(
            nn.Conv1d(1, 16, kernel_size= 2),  # output: [batch, 16, 2]
            nn.ReLU(),
            nn.Flatten(),  # [batch, 16 * 2]
            nn.Linear(16 * 2, 32),
            nn.ReLU(),
            nn.Linear(32, 2)  # Binary classification (output: logits for 2 classes)
        )

    def forward(self, x):
        return self.network(x)

# ITG

# Loading the dataset

In [ ]:
ITG_features_df= pd.read_csv('Dataset/ITG_features_df.csv')

# Stacking

In [ ]:
# Step 1: Extract features and labels
X= ITG_features_df[['F1_sum_vector_magnitude', 'F2_orientation_angle', 'F3_std_magnitude']].values
y= ITG_features_df['Situation'].values

# Step 2: Encode labels ('Fall' → 1, 'Not Fall' → 0)
label_encoder= LabelEncoder()
y_encoded= label_encoder.fit_transform(y)

# Step 3: Split dataset
X_train, X_test, y_train, y_test= train_test_split(X, y_encoded, test_size= 0.2, random_state= 48, stratify= y_encoded)

In [ ]:
# Step 4: Scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

SVM

In [ ]:
# Step 4: SVM
svm= SVC(kernel='rbf', C= 100, gamma= 1, random_state= 48)
svm.fit(X_train_scaled, y_train)

In [ ]:


# -- Model 2: Random Forest --
rf = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=48)
rf.fit(X_train, y_train)

# -- Model 3: XGBoost --
xgb = XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.1,
                    subsample=0.8, use_label_encoder=False, eval_metric='logloss', random_state=48)
xgb.fit(X_train, y_train)

# -- Model 4: CNN --
X_cnn_train = scaler.fit_transform(X_train)
X_cnn_test = scaler.transform(X_test)
X_cnn_train = X_cnn_train.reshape(X_cnn_train.shape[0], 1, X_cnn_train.shape[1])
X_cnn_test = X_cnn_test.reshape(X_cnn_test.shape[0], 1, X_cnn_test.shape[1])

train_loader = DataLoader(CNNFeatureDataset(X_cnn_train, y_train), batch_size=16, shuffle=True)
model_cnn = SimpleCNN(X_cnn_train.shape[2])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_cnn.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_cnn.parameters(), lr=0.001)

model_cnn.train()
for epoch in range(20):
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model_cnn(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

# === 4. Generate Meta-Features ===
def get_predictions(model, X):
    if isinstance(model, nn.Module):
        model.eval()
        with torch.no_grad():
            X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
            outputs = model(X_tensor).cpu().numpy()
            return outputs
    else:
        return model.predict_proba(X)

# For CNN, use reshaped data
X_train_preds = np.column_stack([
    svm.predict_proba(X_train_scaled),
    rf.predict_proba(X_train),
    xgb.predict_proba(X_train),
    get_predictions(model_cnn, X_cnn_train)
])

X_test_preds = np.column_stack([
    svm.predict_proba(X_test_scaled),
    rf.predict_proba(X_test),
    xgb.predict_proba(X_test),
    get_predictions(model_cnn, X_cnn_test)
])

# === 5. Train Meta-Model ===
meta_model = LogisticRegression()
meta_model.fit(X_train_preds, y_train)
final_preds = meta_model.predict(X_test_preds)

# === 6. Evaluate ===
acc = accuracy_score(y_test, final_preds)
print(f"Stacking Ensemble Accuracy: {acc:.4f}")
